In [1]:
import os
from keras.preprocessing import image
categories = ['with_mask','without_mask']
import cv2

In [2]:
categories = ['with_mask','without_mask']

In [3]:
data = []

for category in categories:
  path = os.path.join(category)

  label =  categories.index(category)

  for file in os.listdir(path):
    img_path = os.path.join(path,file)
    img = cv2.imread(img_path)
    img = cv2.resize(img,(224,224))
    
    data.append([img,label])

In [4]:
len(data)

1410

In [5]:
# Shuffling the data so that the system does not develop bias as there is all 705 with mask data at first and 705 without mask data after that.
import random
random.shuffle(data)

In [6]:
X = []
y = []
for features,label in data:
  X.append(features)
  y.append(label)

In [7]:
import numpy as np

In [8]:
X = np.array(X) 
y = np.array(y)

In [9]:
X.shape

(1410, 224, 224, 3)

In [10]:
# Standardize X
X = X/255

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [12]:
from keras.applications.vgg16 import VGG16

In [13]:
vgg = VGG16()

In [14]:
vgg.summary()
# In this VGG model we remove the last layer and add our own custom layer

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [15]:
from keras import Sequential

In [16]:
model = Sequential()

In [17]:
for layer in vgg.layers[:-1]:
  model.add(layer)

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [19]:
for layer in model.layers:
  layer.trainable = False

In [20]:
from keras.layers import Dense

In [21]:
from keras.backend import sigmoid
model.add(Dense(1,activation=sigmoid))

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [23]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [24]:
model.fit(X_train,y_train,epochs=2,validation_data=(X_test,y_test))

Epoch 1/2
36/36 [==============================] - 387s 11s/step - loss: 0.5815 - accuracy: 0.6959 - val_loss: 0.3813 - val_accuracy: 0.9043
Epoch 2/2
36/36 [==============================] - 375s 10s/step - loss: 0.3579 - accuracy: 0.8697 - val_loss: 0.2683 - val_accuracy: 0.9184


In [25]:
cap = cv2.VideoCapture(0) #0 means capture from the webcam of the device

In [ ]:
while True:
  ret, frame = cap.read()

  # call the detection method
  img = cv2.resize(frame,(224,224))
  y_pred = detect_face_mask(img)

#coordinate = detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))

#  for x,y,w,h in coordinate:
#    cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),5)

  if y_pred ==0:
    draw_label(frame,"Mask",(30,30),(0,255,0))
  else:
    draw_label(frame,"No Mask",(30,30),(0,0,255))
  
 #draw_label(frame,"Face Mask Detection",(10,10),(0,0,255))

  cv2.imshow("window",frame)

  if cv2.waitKey(1) & 0xFF == ord('x'):
    break
cv2.destroyAllWindows()

1/1 [==============================] - 0s 357ms/step


1/1 [==============================] - 0s 342ms/step


1/1 [==============================] - ETA: 0s

In [59]:
def detect_face_mask(img):
    
  y_pred = model.predict(img.reshape(1,224,224,3))
  return y_pred[0][0]

In [60]:
def draw_label(img,text,pos,bg_color):
  text_size = cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)

  end_x = pos[0] + text_size[0][0] + 2
  end_y = pos[1] + text_size[0][1] - 2

  cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
  cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)

In [ ]:
# Face detection using HAAR Classifier
#while True:
#  ret,frame = cap.read()
#  coordinate = detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))

#  for x,y,w,h in coordinate:
#    cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),3)

#  cv2.imshow("window",frame)

#  if cv2.waitKey(1) & 0xFF == ord('x'):
#    break
#cv2.destroyAllWindows()

In [ ]:
#haar = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
#def detect_face(img):
#  coordinate = haar.detectMultiScale(img)
#  return coordinate